In [23]:
import tensorflow as tf 
import tensorflow_hub as hub
import tensorflow_text

import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import string

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

from official.nlp import optimization

import matplotlib.pyplot as plt

In [2]:
path =  "E:/LOMBA/Kaggle/Comp/Coommon readilbility/"
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
sample_submission = pd.read_csv(path + 'sample_submission.csv')

STOPWORDS = set(stopwords.words("english"))
def text_cleaning(text):
    text_clean = " ".join([word for word in word_tokenize(text) if not word in STOPWORDS])
    text = re.sub("\s+"," ", text_clean) # remove extra spaces
    text = ''.join([k for k in text if k not in string.punctuation]) # remove punctuation
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text)).strip() # remove unnecessary literals and Perform Case Normalization
    text = ''.join([i for i in text if not i.isdigit()])
    return text

target = train.target.to_numpy()
text = train.excerpt.to_numpy()


In [3]:
txt_clean = []
for word in text:
  txt_clean.append(text_cleaning(word))

In [4]:
train_x, test_x, train_y, test_y = train_test_split(txt_clean, target, test_size=0.2, shuffle=True, random_state=42)
np.array(train_x).shape

(2267,)

In [5]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "http://tfhub.dev/tensorflow/albert_en_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/albert_en_large/3",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 1024].
sequence_output = outputs["sequence_output"]

In [9]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "http://tfhub.dev/tensorflow/albert_en_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/albert_en_base/3",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]

In [10]:
def albert_large_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
    encoder_inputs = preprocessor(text_input)
    outputs = encoder(encoder_inputs)
    pooled_output = outputs["pooled_output"]    
    net = tf.keras.layers.Dropout(0.1)(pooled_output)
    net = tf.keras.layers.Dense(10, activation="relu")(net)
    net = tf.keras.layers.Dense(1, name='regressor')(net)
    model = tf.keras.models.Model(inputs=text_input, outputs=net, name="large_albert")
    return model

In [11]:
large_albert = albert_large_model()
large_albert.summary()

Model: "large_albert"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      {'input_word_ids': ( 0           input_4[0][0]                    
__________________________________________________________________________________________________
keras_layer_3 (KerasLayer)      {'encoder_outputs':  11683584    keras_layer_2[1][0]              
                                                                 keras_layer_2[1][1]              
                                                                 keras_layer_2[1][2]              
_______________________________________________________________________________________

In [13]:
epochs = 5
batch_size = 32
steps_per_epoch = round(len(train_x)//batch_size)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [14]:
callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_root_mean_squared_error', min_delta=0.0001, patience=0, verbose=0,
    mode='min')

In [15]:
large_albert.compile(optimizer=optimizer,
                         loss="mean_squared_error",
                         metrics=tf.keras.metrics.RootMeanSquaredError())

In [32]:
history = large_albert.fit(x=np.array(train_x),
                y=train_y,
                validation_data=(np.array(test_x), test_y),
                epochs=3,
                batch_size= batch_size)


Epoch 1/3
71/71 [==============================] - 942s 13s/step - loss: 0.1478 - root_mean_squared_error: 0.3845 - val_loss: 0.3159 - val_root_mean_squared_error: 0.5620
Epoch 2/3
71/71 [==============================] - 982s 14s/step - loss: 0.1255 - root_mean_squared_error: 0.3543 - val_loss: 0.3159 - val_root_mean_squared_error: 0.5620
Epoch 3/3
71/71 [==============================] - 981s 14s/step - loss: 0.1281 - root_mean_squared_error: 0.3578 - val_loss: 0.3159 - val_root_mean_squared_error: 0.5620


In [19]:
large_albert.evaluate(np.array(test_x), test_y)

18/18 [==============================] - 66s 4s/step - loss: 0.3824 - root_mean_squared_error: 0.6184


[0.3823857307434082, 0.618373453617096]

In [21]:
large_albert.evaluate(np.array(test_x), test_y)

18/18 [==============================] - 71s 4s/step - loss: 0.3200 - root_mean_squared_error: 0.5657


[0.32000213861465454, 0.5656872987747192]

In [34]:
large_albert.evaluate(np.array(test_x), test_y)

18/18 [==============================] - 70s 4s/step - loss: 0.3159 - root_mean_squared_error: 0.5620


[0.31586602330207825, 0.5620195865631104]

In [33]:
large_albert.save("saved_model/3")

INFO:tensorflow:Assets written to: saved_model/3\assets


INFO:tensorflow:Assets written to: saved_model/3\assets


In [44]:
large_albert.save("saved_model/albert_base.h5", save_traces=True)

In [39]:
tf.__version__

'2.5.0'

In [43]:
tf.keras.models.load_model(
    "saved_model/3", custom_objects={"AdamWeightDecay": optimizer}
)

In [47]:
tf.keras.models.load_model(
   "saved_model/albert_base.h5")

TypeError: load_model() got an unexpected keyword argument 'save_traces'